# Conjunto de entrenamiento: Etiquetado de datos

## 1. Creación de dataset de entrenamiento

In [ ]:
import pandas as pd
from pandas import Series, DataFrame

from matplotlib import pyplot as plt

df = pd.read_json('manifest.json')

In [ ]:
df.shape

In [3]:
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source',
       'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted',
       'filter_level', 'lang', 'timestamp_ms', 'retweeted_status',
       'extended_entities', 'possibly_sensitive', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
       'extended_tweet', 'withheld_in_countries'],
      dtype='object')

In [5]:
user = pd.DataFrame([md for md in df.user])

In [6]:
user.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description',
       'translator_type', 'protected', 'verified', 'followers_count',
       'friends_count', 'listed_count', 'favourites_count', 'statuses_count',
       'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang',
       'contributors_enabled', 'is_translator', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'profile_image_url', 'profile_image_url_https', 'default_profile',
       'default_profile_image', 'following', 'follow_request_sent',
       'notifications', 'profile_banner_url'],
      dtype='object')

In [7]:
data = pd.DataFrame(user) 

In [8]:
data.shape

(550000, 39)

In [ ]:
data["isAutomated"] = 0

In [9]:

data['source']= df['source']
data['id2']= df['id']
data['text']= df['text']
data['tweet_created_at']= df['created_at']
data['geo']= df['geo']
data['coordinates']= df['coordinates']
data['place']= df['place']
data['reply_count']= df['reply_count']
data['retweet_count']= df['retweet_count']
data['favorite_count']= df['favorite_count']
data['favorited']= df['favorited']
data['retweeted']= df['retweeted']
data['possibly_sensitive']= df['possibly_sensitive']
data['lang']= df['lang']
data['timestamp_ms']= df['timestamp_ms']

In [10]:
for p in data.iterrows():
    if ("bot" in p[1]['source'] or "tweetdeck" in p[1]['source'] or "hootsuite" in p[1]['source'] or "pulse" in p[1]['source'] or "buffer" in p[1]['source'] or "agorapulse" in p[1]['source'] or "peing" in p[1]['source'] or "dlvrit" in p[1]['source'] or "zapier" in p[1]['source'] or "stafsmoketest" in p[1]['source'] or "lithium" in p[1]['source'] or "IFTTT"in p[1]['source']):
        
        data["isAutomated"].iloc[p[0]] = 1

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## 2. Creación de árbol de decisión a mano

In [11]:
##Primer split: ¿El tweet proviene de una fuente automática?
derecha = pd.DataFrame()
izquierda = pd.DataFrame()
def primer_split(data):
    derecha = (data.loc[data["isAutomated"] == True] )
    izquierda = (data.loc[data["isAutomated"] == False] )
    derecha
    return derecha;

In [12]:
##Second split: ¿La cuenta del tweet esta verificada?

def second_split(derecha): 
    derecha = pd.DataFrame(derecha.loc[derecha['verified'] == False] )
    izquierda = pd.DataFrame(derecha.loc[derecha['verified'] == True] )
    return derecha

In [13]:
##Third split: ¿Hay mucha diferencia entre seguidores y seguidos?
def third_split(derecha
    derecha = pd.DataFrame(derecha.loc[ derecha['followers_count']- derecha['friends_count'] > 15000] )
    izquierda = pd.DataFrame(derecha.loc[ derecha['followers_count']- derecha['friends_count'] < 1000] )
    return derecha

In [14]:
##Fourth split: ¿La cuenta tweetea mucho?
##Find best split
isBot = pd.DataFrame()
NotBot = pd.DataFrame()
def fourth_split(derecha):
    isBot = pd.DataFrame(derecha.loc[ derecha['statuses_count'] > 10000] )
    NotBot = pd.DataFrame(derecha.loc[ derecha['statuses_count']< 10000] )
    BotNames = isBot['screen_name']
    return isBot

In [15]:
##¿Cómo se llaman los usuarios de las cuentas?
def find_names(isBot):
    names = isBot['screen_name'].unique()
    return names

In [16]:
##Si ejecutamos todo junto obtenemos los nombres que debemos estudiar si verdaderamente son bots
find_names(fourth_split(third_split(second_split(primer_split(data)))))

array(['joniele13', 'AsterloreRB', 'celebritykimdot', 'fuwafuwanokuma',
       'marisascruz', 'INTLGLOBALrp', 'OnetWiadomosci', 'therakyatpost',
       'ModiforPMOrg', 'NinoProdigio', 'Whalepool_Labs', 'Tenerife_Ocio',
       'MamaWikwik', 'nnaaiiff5', 'boshi3', 'Medi1TV', 'GarudaCares',
       'JoeyTribbiiani', 'itz_pandian', 'ichio_x', 'Law_Effect_Bot',
       'Ansa_Piemonte', 'Satomi_pic', 'mita___21', 'NewsBollywood',
       'DiarioBitcoin', 'wccftechdotcom', 'BawaHS', 'gloomynews',
       'machinelearnflx', 'AnxietyUK', 'VysotskyQuote', 'pikachu_pcn',
       'SocCivilMx', 'aman122322', 'bundatatan', 'PRFMnews',
       'ScienceAlert', 'disney_home', 'NUFCThreatLevel', 'Charlotte__77',
       'MandyRosedotCOM', 'barqnewskw', 'basquetplus', 'wvgazettemail',
       'rsumen', 'EbruTVKenya', 'BurganNewsKW', 'rutadelmaule',
       'jockosrocket', 'TasbihIstighfar', 'breakcuffsegy', 'Mag_HR',
       'agripulse', 'GetVideoBot', 'Droid_News', 'NANCYPAZOS',
       'jeffbullas', 'mugtama', 'a

In [17]:
data.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description',
       'translator_type', 'protected', 'verified', 'followers_count',
       'friends_count', 'listed_count', 'favourites_count', 'statuses_count',
       'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang',
       'contributors_enabled', 'is_translator', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'profile_image_url', 'profile_image_url_https', 'default_profile',
       'default_profile_image', 'following', 'follow_request_sent',
       'notifications', 'profile_banner_url', 'isAutomated', 'source', 'id2',
       'text', 'tweet_created_at', 'geo', 'coordinates', 'place',
       'reply_count', 'retweet_count', 'favorite_count', 'favorited',
      

In [18]:
data["isBot"] = 0

## 3. Etiquetado de datos: columna "isBot"

In [18]:
for p in data.iterrows():
    if ("KangHen12" in p[1]['screen_name'] or "TAKAHIRO_EXbot" in p[1]['screen_name'] or "loliconder" in p[1]['screen_name'] or "KakGati" in p[1]['screen_name'] or "oppai_tanima1" in p[1]['screen_name'] or "tokutaro" in p[1]['screen_name'] or "TwitAA_bot" in p[1]['screen_name'] or "feb_feby91" in p[1]['screen_name'] or "futomomonoero" in p[1]['screen_name'] or "GetVideoBot" in p[1]['screen_name'] or "AbhiQeep" in p[1]['screen_name'] or "dognkitty"in p[1]['screen_name'] or "benymd_bot" in p[1]['screen_name'] or "m_almutairi" in p[1]['screen_name'] or "school_aruaru" in p[1]['screen_name'] or "ThahaKhabar" in p[1]['screen_name'] or "ginntama0525" in p[1]['screen_name'] or "veganfuture" in p[1]['screen_name'] or "DrLoverLova" in p[1]['screen_name'] or "yu___ichi07" in p[1]['screen_name'] 
        or "jatirado" in p[1]['screen_name'] 
        or "adutube" in p[1]['screen_name'] 
        or "few7432" in p[1]['screen_name'] 
        or "hiro_ta_so" in p[1]['screen_name'] 
        or "VysotskyQuote" in p[1]['screen_name'] 
        or "AbhiQeep" in p[1]['screen_name'] 
        or "waha1950"in p[1]['screen_name']):
        
        data["isBot"].iloc[p[0]] = 1

In [44]:
data['isBot'].unique()

array([0, 1])


## 4. Transformaciones de atributos string a number para aplicar ML

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(data["verified"])
print(transfomed_label)

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data['verified'].unique())
le.classes_
vals = le.transform(data['verified'])
vals

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
data['verified_transform']= vals

In [22]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data['source'].unique())
le.classes_
vals = le.transform(data['source'])
vals

array([1740,  684, 1740, ...,  685, 1740,  685])

In [23]:
data['source_transform']= vals


In [46]:
data['source_transform'].value_counts()

367    90720
368    80848
938    21265
357     3893
349     2822
       ...  
828        1
572        1
444        1
316        1
0          1
Name: source_transform, Length: 1511, dtype: int64

## 5. Guardado del Dataset

In [ ]:
data.to_csv('TrainBot.csv',encoding='utf-8', index=False)

In [ ]:
data.to_json('TrainBot.json')